This code is usage of viewer shared by following discussion.
https://www.kaggle.com/competitions/eedi-mining-misconceptions-in-mathematics/discussion/542654

I can't seem to get streamlit to work in the Kaggle notebook environment, so I'll just show the steps here. To actually use it, you need to run it in your local environment or colab environment.

In [1]:
%%capture
!pip install streamlit

In [2]:
%%writefile visualizer.py

from pathlib import Path

import polars as pl
import streamlit as st
from streamlit.delta_generator import DeltaGenerator

# TODO: set your competition data path
INPUT_DIR = Path("/kaggle/input/eedi-mining-misconceptions-in-mathematics")


def replace_latex(text: str) -> str:
    return text.replace(r"\[", "$").replace(r"\]", "$").replace(r"\(", "$").replace(r"\)", "$")


def plot_answer(
    col: DeltaGenerator,
    answer: str,
    misconception_id: float | None,
    alphabet: str,
    is_correct_answer: bool,
    misconception_mapping: dict[int, str],
) -> None:
    if misconception_id is not None:
        misconception_id = int(misconception_id)
        misconception_name = misconception_mapping[misconception_id]
    else:
        misconception_name = ""

    with col:
        if is_correct_answer:
            st.write(f"{alphabet} (Correct Answer)")
            st.success(f"Answer: {replace_latex(answer)}")
            st.success(f"Misconception: {misconception_name} (id={misconception_id})")
        else:
            st.write(alphabet)
            st.error(f"Answer: {replace_latex(answer)}")
            st.error(f"Misconception: {misconception_name} (id={misconception_id})")


def main() -> None:
    st.set_page_config(layout="wide")
    st.title("Eedi visualizer")

    df = pl.read_csv(INPUT_DIR / "train.csv")
    misconception_mapping = pl.read_csv(INPUT_DIR / "misconception_mapping.csv")
    misconception_mapping = dict(
        zip(misconception_mapping["MisconceptionId"].to_list(), misconception_mapping["MisconceptionName"].to_list())
    )
    with st.expander("Misconception Mapping"):
        st.write(misconception_mapping)

    with st.sidebar:
        st.markdown("## Filter")
        # id = st.selectbox(label="QuestionId", options=df["QuestionId"].to_list())
        id = st.number_input(label="QuestionId", min_value=0, max_value=df["QuestionId"].max())
    _data = df[id].to_dict(as_series=False)
    data = {k: v[0] for k, v in _data.items()}

    st.info(f"ConstructName: {data['ConstructName']}")
    st.info(f"SubjectName: {data['SubjectName']}")
    st.info(f"QuestionText: {replace_latex(data['QuestionText'])}")

    col1, col2, col3, col4 = st.columns(4)
    plot_answer(
        col1, data["AnswerAText"], data["MisconceptionAId"], "A", data["CorrectAnswer"] == "A", misconception_mapping
    )
    plot_answer(
        col2, data["AnswerBText"], data["MisconceptionBId"], "B", data["CorrectAnswer"] == "B", misconception_mapping
    )
    plot_answer(
        col3, data["AnswerCText"], data["MisconceptionCId"], "C", data["CorrectAnswer"] == "C", misconception_mapping
    )
    plot_answer(
        col4, data["AnswerDText"], data["MisconceptionDId"], "D", data["CorrectAnswer"] == "D", misconception_mapping
    )


if __name__ == "__main__":
    main()


Writing visualizer.py


In [ ]:
# # Run the following command to start the application. 
# # It is commented out because it is always executed in the kaggle environment.

# !streamlit run visualizer.py




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.19.2.2:8501
  External URL: http://35.244.103.46:8501



As I said at the beginning, this URL is not accessible. When actually running the app, please run it in your local environment or colab environment and follow the link displayed to access the app.